<a id="header-notebook"></a>
![](https://raw.githubusercontent.com/AdrianoPereira/adrianopereira.github.io/master/assets/banner_notebook.png)

# Exemplo de aplicação do algoritmo de regressão linear

**Instrutores**: Adriano Almeida, Felipe Carvalho e Felipe Menino

**Realização**: Dia 15/09


**Descrição**:  Este notebook tem como próposito apresentar a aplicação do algoritmo de regressão linear no conjunto de dados [*World Happiness Report 2020*](https://worldhappiness.report/ed/2020) utilizando a biblioteca [*scikit-learn*](https://scikit-learn.org) na linguagem de programação [Python](http://python.org).


**Sumário**:
* [Descrição do conjunto de dados](#description-dataset)
* [Descrição da atividade](#description-task)
* [Importação das bibliotecas](#import)
* [Carregando base de dados](#load-dataset)
* [Análise exploratória dos dados](#eda)
* [Separando conjunto de dados](#split)
* [Aplicando a máquina de vetores de suporte](#model)
* [Avaliação do modelo](#evaluation)
* [Agora é com você](#homework)
* [Referências](#references)


**Links Úteis**:
* [Livro **Introdução ao Machine Learning**](https://dataat.github.io/introducao-ao-machine-learning/)
* [Exemplo de **Classificação** em Python](https://www.kaggle.com/phelpsmemo/intro-ml-python-knn-worcap2020)
* [Exemplo de **Regressão com máquinas de vetores de suporte** em Python](https://www.kaggle.com/lordadriano/intro-ml-python-svr-worcap2020)
* [Exemplo de **Agrupamento Hierquico** em R](https://www.kaggle.com/oldlipe/intro-ml-r-hiererquico-worcap2020)

> **Atenção**: Este documento foi redigido apenas para fins didáticos, sendo assim, as técnicas e conjunto de dados utilizados aqui podem e devem ser evoluídos em caso de sua continuidade com cunho científico.

## Descrição do conjunto de dados <a id="description-dataset"></a> [^](#header-notebook)
<hr />

![](https://raw.githubusercontent.com/AdrianoPereira/introducao-ao-machine-learning/master/src/assets/happiness-reg.png)

O conjunto de dados utilizado neste notebook é o World Happiness Report 2020 publicado por Helliwell et al. (2020). O primeiro World Happiness foi apresentado em 2012 na Assembléia Geral das Nações Unidas. Este é o oitavo relatório, e assim como os anteriores, busca fornecer informações sobre o estado de felicidade e bem-estar da população em diversos países ao redor do mundo. Este conjunto de dados contém informações de 153 países, provenientes principalmente dos atributos descritos a seguir:

- **Country name**: Nome do país onde foram obtidas as informações.
- **Logged GDP per capita**: Indicador econômico relacionado à renda per capita.
- **Social support**: Sensação de acolhimento de amigos e familiares em momentos de dificuldade e/ou lazer.
- **Healthy life expectancy**: Expectativa de vida da população do país.
- **Freedom to make life choices**: Sensação de que todo cidadão possa ter liberdade para fazer qualquer coisa.
- **Generosity**: Sensação de compaixão e autruísmo das pessoas.
- **Perceptions of corruption**: Sensação de confiança no governo, entidades e empresas.
- **Ladder score**: Índice de felicidade baseado nos atributos anteriores.
![](http://)- **Dystopia + residual**: Métrica de comparação com valores mais baixos de todos os atributos.

## Descrição da atividade <a id="description-task"></a> [^](#header-notebook)
<hr />

Neste exemplo será aplicado uma regressão linear simples, utilizando apenas duas variáveis. A variável independente ($x$) será o índice de felicidade, já a variável dependente ($y$) será escolhida baseado em uma superficial análise exploratória dos dados para identificar o atributo que mais se associa a ele. Após ser escolhida a variável dependente, o conjunto de dados será separado em grupos de treinamento e teste, que em seguida serão submetidos ao modelo de regressão linear utilizando a biblioteca *scikit-learn*. Ao final serão aplicadas algumas métricas de avaliação do modelo e visualização dos resultados.

> **Aviso**: Para a maioria das atividades que envolvem algoritmos de aprendizagem de máquina, é recomendado que haja uma certa dedicação na realização de uma análise exploratória aprofundada, principalmente se os dados não forem muito bem conhecidos. A análise exploratória dos dados pode ajudar a ter uma visão geral rápida a respeito do *dataset* que está sendo trabalhado. Para fins didáticos, a análise exploratória dos dados não será muito aprofundada aqui.

## Importação das bibliotecas <a id="import"></a> [^](#header-notebook)
<hr />

Antes de mais nada, devemos importar as bibliotacas que iremos utilizar durante a atividade. Será utilizado o `Pandas` para o carregamento da base de dados, módulos do `Sklearn` para a divisão dos dados e para o o modelo de regressão linear e por fim, as bibliotecas `Matplotlib` e `Seaborn` para a exibição dos gráficos.


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import math
from sklearn import metrics
from sklearn import linear_model
import matplotlib.pyplot as plt
import seaborn as sns

## Carregando base de dados <a id="load-dataset"></a> [^](#header-notebook)
<hr />

O bloco a seguir mostra o trecho que faz a importação da base de dados e exibe seu cabeçalho.

> **Dica**: O Kaggle permite o envio de base de dados próprias para o projeto e também a importação dos *datasets* públicos disponíveis na plataforma.

In [ ]:
df = pd.read_csv('/kaggle/input/world-happiness-report/2020.csv')
df.head()


O conjunto de dados possui algumas colunas extras, que são extensões dos atributos principais, que não serão necessárias para este exemplo. Sendo assim, serão selecionados os registros somente das colunas que será utilizada. Existem várias formas de filtrar as colunas em um `Pandas DataFrame`, aqui será utilizado uma lista com seus nomes, conforme mostrado no bloco a seguir:

In [ ]:
columns = ['Country name', 'Ladder score', 'Logged GDP per capita', 'Social support', 
           'Healthy life expectancy', 'Freedom to make life choices', 'Generosity',
           'Perceptions of corruption', 'Dystopia + residual']
new_df = df[columns]
new_df.head()

## Análise exploratória dos dados <a id="eda"></a> [^](#header-notebook)
<hr />

O `Pandas DataFrame` possui alguns métodos que podem auxiliar no início da exploração dos dados. O método `describe` mostra algumas informações estatísticas a repeito dos dados númericos, conforme mostrado a seguir:

In [ ]:
new_df.describe()

Outro recurso interessante para ser utilizado na exploração dos dados são as correlações entre os atributos, já que as técnicas de regressão estão diretamente associadas a elas. Com poucas linhas de código é possível obser um gráfico com a matriz de correlação das variáveis, como mostrado a seguir:

In [ ]:
plt.figure(figsize=(10, 10))
sns.heatmap(new_df.corr(), annot=new_df.corr(), cmap='Reds')
plt.show()

Como pode ser observado no gráfico da figura acima, é possível identificar de forma lúcida as variáveis mais correlacionadas. Essas relações são observadas através da diagonal superior ou inferior da matriz de correção.
Outra ferramenta que pode auxiliar na visualização da correlações das variáveis é o gráfico de dispersão (*scatter plot*). No trecho a seguir é executado o comando para visualizar o gráfico de disperções, entre todas as variáveis.

In [ ]:
sns.pairplot(new_df)
plt.show()

No gráfico acima as relações entre as variáveis ficam bem mais destacas, por exemplo, os atributos que representam a renda per capita, suporte social e a expectativa de vida possuem são os que possuem maior correlação positiva com o índice de felicidade, sendo assim, eles são fortes candidatos a serem variáveis dependentes. Diante disso, o atributo que indica o PIB percapta será utilizado neste exemplo para ser a variável dependente.

> **Importante**: Nem sempre a alta correlação entre as variáveis pode indicar necessariamente que elas estejam associadas, isso pode ser uma causalidade. 

## Separando conjunto de dados <a id="split"></a> [^](#header-notebook)
<hr />

Durante as tarefas que envolvem treinamento e testes de modelos de aprendizado de máquina, é altamente recomendado que o conjunto de dados disponível seja separado em grupos de treinamento e teste ou treinamento, teste e validação. A divisão do conjunto de dados evita que o modelo fique enviezado, ou seja, que ele fique especialista somente nos dados aos quais ele foi treinado e perca a sua capacidade de generalização, conhecido como *overfitting*. Neste exemplo, não há necessidade de ser utilizado o conjunto de validação, pois para preparar o modelo será utilizado apenas conjunto de treinamento e a avaliação com o conjunto de testes. A criação do terceiro grupo de dados (validação) pode ser utilizados quando o modelo será testado durante o treinamento.

Antes de realizar a divisão do conjunto de dados, serão criadas listas separadas para armazenas as informações de cada atributo que será utilizado neste exemplo, conforme mostrado no trecho de código a seguir:

In [ ]:
x = new_df['Ladder score'].values
y = new_df['Logged GDP per capita'].values

Os valores do índice de felicidade foram adicionados na variável `x` e os valores referentes ao atributo de PIB per capta foram adicionados na variável `y`. Embora neste exemplo será utilizado somente uma variável independente, o modelo de regressão linear pode ter mais. Sendo assim, é necessário realizar mais um preprocessamento nos dados, transformando a variável independente em uma matriz, já que o modelo espera os dados de entrada nesse formato, e não como uma lista, como eles estão. O trecho a seguir faz a transforação da lista em uma matriz:

In [ ]:
x = x.reshape((-1, 1))

Para a separação dos conjuntos de dados em grupos de treinamento e teste, será utilizada a função `train_test_split`, disponível no `sklearn`, conforme mostrado no trecho a seguir:

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.15, random_state=0)

O parâmetro `test_size` da função `train_test_split` indica a porcentagem de dados que será reservada para o grupo de teste e o atributo `random_state` é para garantir que a função sempre irá pegar a mesma amostra aleatória de dados para cada grupo.

## Aplicando a regressão linear <a id="model"></a> [^](#header-notebook)
<hr />

Um vez realizados as análises e preprocessamentos dos dados, a regressão poderá ser aplicada. O trecho a seguir, cria e treina o modelo de regressão linear:

In [ ]:
model = linear_model.LinearRegression().fit(x_train, y_train)

A seguir são exibidos os coeficientes $\alpha$ e $\beta$ da reta da equação da regressão linear:

In [ ]:
alpha, beta = model.coef_[0], model.intercept_
print('Alpha = ', alpha)
print('Beta = ', beta)

Uma vez que o modelo foi treinado, podemos exibir a reta da equação da regressão linear, como mostrado no exemplo a seguir: 

In [ ]:
y_pred_train = model.predict(x_train)

plt.figure(figsize=(10, 7))
plt.scatter(x_train, y_train, color='blue', edgecolor='k', alpha=.75, label="Dados de treinamento")
plt.plot(x_train, y_pred_train, color='red', linewidth=1, label=r"$y_{i}$ = $\alpha+\beta x_{i}$")
plt.xlabel('Índice de felicidade')
plt.ylabel('PIB per capta')
plt.legend()
plt.show()

Agora podemos adicionar os dados de teste também para comparar com os dados de treinamento.

In [ ]:
y_pred_test = model.predict(x_test)

plt.figure(figsize=(10, 7))
plt.scatter(x_train, y_train, color='blue', edgecolor='k', alpha=.75, label="Dados de treinamento")
plt.scatter(x_test, y_test, color='green', edgecolor='k', alpha=.75, label="Dados de teste")
plt.plot(x_train, y_pred_train, color='red', linewidth=1, label=r"$y_{i}$ = $\alpha+\beta x_{i}$")
plt.xlabel('Índice de felicidade')
plt.ylabel('PIB per capta')
plt.legend()
plt.show()

## Avaliação do modelo <a id="evaluation"></a> [^](#header-notebook)
<hr />
Além de auxiliarem na análise exploratória dos dados, os gráficos também podem ser uma ótima ferramenta para análise dos resultados. A seguir é mostrado um gráfico de dispersão dos dados reais em função dos dados previstos pela regressão linear.

In [ ]:
plt.figure(figsize=(10, 7))
plt.scatter(y_test, y_pred_test, color='k', edgecolor='k', alpha=.75)
plt.xlabel(r'$y_{i}$ real')
plt.ylabel(r'$y_{i}$ previsto')
plt.plot([min(y_test), max(y_test)], [min(y_pred_test), max(y_pred_test)], label='Diagonal secundária', linestyle='--', alpha=.75)

plt.legend()
plt.show()

Os modelos de aprendizado de máquina devem ter uma capacidade de generalização razoável, sendo assim, dificilmente acertarão os valores exatos, principalmente em tarefas de regressão. O gráfico de dispersão acima tem como intuito mostrar visualmente a correlação entre os dados observados e os previstos pela regressão. Como se trata de uma regressão linear, e os dados não possuem uma correlação perfeita, é esperado que haja esse pequeno desvio em relação à diagonal secundária.

As métricas de erro mais utilizadas para se avaliar os métodos de regressão são o erro médio absoluto (MAE - sigla do inglês, *mean absolute error*), erro quadrático médio (MSE - sigla do inglês, *mean square error*) e a raiz do erro quadrático médio (RMSE - sigla do inglês, *root mean square error*), Essas e outras métricas estão implementadas no `sklearn`. O MAE, como o próprio nome sugere, está relacionado à média dos erros absolutos entre os valores reais e os previstos, obtido pela soma das diferenças entre eles. O MSE é a média da diferença entre os valores reais e previstos elevado ao quadrado. Já o RMSE é a raiz do erro médio quadrático médio entre os valores previstos e os valores reais. No trecho a seguir são apresentadas as três métricas mencionadas:

In [ ]:
print("MAE: %.3f"%metrics.mean_absolute_error(y_test, y_pred_test))
print("MSE: %.3f"%metrics.mean_squared_error(y_test, y_pred_test))
print("RMSE: %.3f"%math.sqrt(metrics.mean_squared_error(y_test, y_pred_test)))


## Agora é com você <a id="homework"></a> [^](#header-notebook)
<hr />

> Um das principais motivações da geração do World Happiness Report é avaliar o bem estar e qualidade de vida das pessoas nos países onde moram, será que a felicidade pode está associada à expectativa de vida dos países?

📚 Como sugestão de atividade, aplique a técnica apresentada para descobrir as relações lineares mencionada na questão acima.

## Referências <a id="references"></a> [^](#header-notebook)

Helliwell, John F., Haifang Huang, Shun Wang, and Max Norton. "Social environments for world happiness." World Happiness Report 2020 (2020).